In [134]:
import pandas as pd
import numpy as np

In [137]:
df = pd.read_csv('resources/dataset_train.csv', index_col=0)
df.head()

,Hogwarts House,First Name,Last Name,Birthday,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
Index,,,,,,,,,,,,,,,,,,
0,Ravenclaw,Tamara,Hsu,2000-03-30,Left,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89
1,Slytherin,Erich,Paredes,1999-10-14,Right,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45
2,Ravenclaw,Stephany,Braun,1999-11-03,Left,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42
3,Gryffindor,Vesta,Mcmichael,2000-08-19,Left,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64
4,Gryffindor,Gaston,Gibbs,1998-09-27,Left,60158.0,436.775204,-7.820623,NaN,2.236,-444.262537,599.324514,-3.444377,937.434724,4.311066,-0.264070,-256.38730,157.98


In [144]:
target = 'Hogwarts House'

courses = ['Arithmancy',
            'Astronomy',
            'Herbology',
            'Defense Against the Dark Arts',
            'Divination',
            'Muggle Studies',
            'Ancient Runes',
            'History of Magic',
            'Transfiguration',
            'Potions',
            'Care of Magical Creatures',
            'Charms',
            'Flying']

X = df[courses]
y = df[target]
X.shape, y.shape

((1600, 13), (1600,))

In [153]:
X = X.fillna(0)

In [158]:
n, m = X.shape

In [187]:
w = np.zeros((m, 1))
b = np.zeros((1))

In [188]:
Xv = X.values

In [189]:
z = Xv @ w + b

In [190]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [196]:
y_hat = sigmoid(z)
y_hat = y_hat.ravel()

In [197]:
y_true = y.apply(lambda x: 1 if x == 'Ravenclaw' else 0)

In [198]:
def log_loss(y_true, y_hat):
    return -(y_true * np.log(y_hat) + (1 - y_true) * np.log(1 - y_hat)).mean()

In [218]:
err = (y_hat - y_true).values.reshape(-1, 1)

In [219]:
dw = (np.multiply(err, Xv)).mean(axis=0)
dw

array([ 1.09703084e+04,  1.49712281e+02, -8.00297056e-01, -1.50009208e+00,
        1.88667812e-01, -2.40981651e+02,  7.95648753e+01,  1.14421482e-01,
        2.19856978e+02,  1.01070364e+00, -2.70960613e-02, -5.77067699e+01,
        1.20809125e+01])

In [222]:
db = err.mean(axis=0)
db

array([0.223125])

In [223]:
lr = .05

In [232]:
Xv = (Xv - Xv.mean(axis=0)) / (Xv.std(axis=0))

In [234]:
np.random.seed(42)
w = np.random.randn(m, 1).ravel() * 0.01
b = np.zeros((1))

iter = 100
for i in range(iter):
    z = Xv @ w + b
    y_hat = sigmoid(z)
    y_hat = y_hat.ravel()
    print(f'[{i}]: {log_loss(y_true, y_hat)}')
    err = (y_hat - y_true).values.reshape(-1, 1)
    dw = np.multiply(err, Xv).mean(axis=0)
    db = err.mean(axis=0)
    
    w = w - lr * dw
    b = b - lr * db

[0]: 0.6858635633072413
[1]: 0.6549966171298763
[2]: 0.6267007296654237
[3]: 0.600725055257914
[4]: 0.5768386264159691
[5]: 0.5548312258989128
[6]: 0.5345131933172117
[7]: 0.515714520195702
[8]: 0.49828351515462077
[9]: 0.48208524907175354
[10]: 0.46699992761149534
[11]: 0.452921288520437
[12]: 0.4397550834877685
[13]: 0.42741767747556425
[14]: 0.41583478005757
[15]: 0.40494031135024117
[16]: 0.39467539777682176
[17]: 0.3849874887562274
[18]: 0.375829583393019
[19]: 0.367159555600225
[20]: 0.35893956628206314
[21]: 0.3511355518793463
[22]: 0.3437167795002575
[23]: 0.3366554598728492
[24]: 0.3299264103711747
[25]: 0.3235067613314303
[26]: 0.31737569976026636
[27]: 0.31151424533290456
[28]: 0.3059050542820431
[29]: 0.3005322473935787
[30]: 0.29538125885873806
[31]: 0.2904387031924691
[32]: 0.28569225782334806
[33]: 0.281130559299012
[34]: 0.2767431113408117
[35]: 0.2725202032288328
[36]: 0.2684528372097167
[37]: 0.26453266380010454
[38]: 0.26075192401260633
[39]: 0.2571033976628793
[40]: